# SQLite3 Database


In this lesson, you will learn about SQLite3, Python DB-API 2.0 interface for SQLite3 databases.

SQLite3 is a C language library that provides lightweight disk-based database and can be accessed using SQL query language. SQLite3 can be used for internal data storage application and also to prototype an application then port the code in larger databases such as PostgreSQL or Oracle.


**SQL** (**Structured Query Language**) is a standard language for accessing and manipulating databases. It is mainly used to create tables in the database and to query (or ask) the database about the data stored to retrieve or modify that data.

Suppose you want to design a database of a bank. The main structures of any database are the tables that hold information.  You can use the SQL language to access the database and perform different operations on it. For instance, you can create a **table called clients** which stores information about differnt clients. This table will have columns such as **full_name**, **address**, **account_no**, **balance**, ... etc. You may also want to access and modify the table in different ways. Here are some examples of table operations:

- Retrive records from the clients table, for example selecting clients who have a balance less than \$1000. 

- Update some clients' records, for example to add $100 to his current balance or change his address.

- Delete some clients' records who have closed their accounts.

- Add new clients records who recently joined the bank.

and so on.

The queries are operations done using SQL commands to access and manipulate the database and the most common queries are:

 - CREATE TABLE: to create a new table in the database.
 
 
 - INSERT: to add new row (record) in the table.
 
 
 - SELECT: to select a particular row(s) in the table.
 
 
 - DELETE: to remove row(s) from the table.
 
 
 - UPDATE: to update row(s) in the table.


## Open connection to DB using SQLite3

To create a database, import the sqlite3 module and use it to call connect() method that creates a **connection** object to your database. The database will be saved in a file with .db extension in your computer.

In the example below, we imported sqlite3 module and called connect() to create a database object **conn**. The DB is saved  in **example.db** file. Try to run the following cell and check if a file with name example.db created in your current directory.


In [1]:
import sqlite3

# create a db in a .db file
conn = sqlite3.connect('example.db')

**NOTES**: 

- If you want to open example.db, try to open it with a text editor. Since our database type is SQLite3, the file will start with "SQLite format 3".


- To create a temporary database in the computer RAM, use the special name **:memory:**. The DB in this case will not be saved in a file on your disk-drive. This way you don't need to pass a database file name to connect() because your database is stored only in the memory.

In [3]:
# create a db in the computer RAM
conn = sqlite3.connect(':memory:')

When we are done working with the DB we need to close the connection.

In [2]:
# close the database
conn.close()

## SQL commands - query the database

After making a connection with the database and before making any operation with the database we need to create a **cursor object**. Imagine this cursor object as a pointer to the data you want to access or manipulate in the database. Pass the SQL statements to the cursor object to execute them using the **execute()** method. Finally it is necessary to commit the changes. 

In the example below, we are going to create a products table with name, quantity, and price columns.

In [3]:
# create a connection to the DB since we closed it last time
conn = sqlite3.connect('example.db')

# create cursor object
c = conn.cursor()

# create a table products in database example.db
# the table has 3 columns, name, quantity and price
c.execute('''CREATE TABLE products(name text, quantity integer, price real)''')

# insert a row of data (record) in the table
c.execute("INSERT INTO products VALUES ('Laptop', 10, 849.99)")

# save (commit) the changes
conn.commit()

# close the connection if we are done with your DB
# make sure ALL changes have been committed or they will be lost.
conn.close()

Note that the data you have saved using **commit()** method is persistent and is available in subsequent sessions.

## Using Python's variables to perform SQL queries

You may want to do SQL operations using values from Python variables. But writing a query using Python’s strings is insecure; it makes your program vulnerable to an SQL injection attack.

**Solution**: use the DB-API parameter substitution. 

### How to do that ?!

Put **?** as a placeholder wherever you want to use a value, and then provide a tuple of values as the second argument to the cursor’s execute() method. 

In this example, we try to **select** all the records in the products table where the product name is Laptop. We show both the insecure and secure methods of writing SQL operation using Python variables. Also, we used the method **fetchone()** that will fetch only one record from the set of records selected. Finally, we used **executemany()** method to **insert** many products in the table at a time.

In [2]:
import sqlite3

conn = sqlite3.connect('example.db', timeout=10)

c = conn.cursor()

#------------------------------
# don't do that --> INSECURE!
#------------------------------
name = 'Laptop'
c.execute("SELECT * FROM products WHERE name = '%s'" % name)

#------------------------------
# do this instead --> SECURE
# DB-API parameter substitution
#------------------------------
t = ('Laptop',)

# second argument is tuple t
c.execute('SELECT * FROM products WHERE name=?', t)

# fetch single matching row
print(c.fetchone())

# larger example that inserts many records at a time
new_products = [('Disk drive', 30, 44.99), ('Printer', 50, 175.00), ('Tablet', 20, 90.00)]

# this method returns the cursor 
c.executemany('INSERT INTO products VALUES (?,?,?)', new_products)


('Laptop', 10, 849.99)


**NOTE**:

Sometimes you see a file named **example.db-journal** in the same directory as your database file. This is called a rollback journal file and is always located in the same directory as the database file and has the same name as the database file but with extension **db-journal**. 
- The rollback journal is usually created when a transaction is first started and is usually deleted when a transaction commits or rolls back. 
- The rollback journal file is essential for implementing the commit and rollback capabilities of SQLite3. 
- Without a rollback journal, SQLite3 would be unable to rollback an incomplete transaction, and if a crash or power loss occurred in the middle of a transaction the entire database would likely go corrupt without a rollback journal.


Expect a file with the name **example.db-journal** created in your current directory. If you run the code below, the journal file will be deleted.

In [7]:
conn.commit()
conn.close()

To retrieve data after executing a **SELECT** statement, you can either treat the cursor as an iterator, call the cursor’s fetchone() method to retrieve a single matching row, or call fetchall() method to get a list of all matching rows.

This example uses the iterator form:

In [6]:
# iterate over all the rows in table products
# in ascending order based on the price
# cheapest item is printed first

for row in c.execute('SELECT * FROM products ORDER BY price'):
    print(row)

('Disk drive', 30, 44.99)
('Tablet', 20, 90.0)
('Printer', 50, 175.0)
('Laptop', 10, 849.99)


### Conclusion

In this lesson we have learned how to create a simple database stored in .db file. We also discussed how to make a connection to the database and how to query the database using SQL commands such as CREATE, INSERT, and SELECT. 

Next, we will discuss the relationship between SQLite3 and Python types.